In [1]:
import collections
import re

In [2]:
import aocd

In [3]:
rules = aocd.get_data(day=7, year=2020).splitlines()
len(rules)

594

In [4]:
rules[4]

'vibrant fuchsia bags contain 2 dark lime bags, 2 mirrored black bags, 2 light magenta bags, 2 drab chartreuse bags.'

In [5]:
def parse_rules(rules):
    for line in rules:
        adj, color, *ignore = line.split(' ')
        bag = f'{adj} {color}'
        holds = re.findall('(\d) ([a-z ]*) bag', line)
        yield bag, holds

In [6]:
list(parse_rules(rules))[:5]

[('clear maroon', [('1', 'dull lavender')]),
 ('wavy turquoise',
  [('4', 'vibrant magenta'),
   ('4', 'light violet'),
   ('5', 'bright gold'),
   ('2', 'faded black')]),
 ('wavy beige',
  [('3', 'plaid magenta'),
   ('3', 'wavy lime'),
   ('2', 'clear turquoise'),
   ('3', 'muted cyan')]),
 ('mirrored black',
  [('1', 'plaid red'), ('3', 'light gold'), ('3', 'wavy violet')]),
 ('vibrant fuchsia',
  [('2', 'dark lime'),
   ('2', 'mirrored black'),
   ('2', 'light magenta'),
   ('2', 'drab chartreuse')])]

### Solution to Part 1

In [7]:
def may_be_inside(rules):
    dd = collections.defaultdict(list)
    for bag, holds in parse_rules(rules):
        for hold in holds:
            dd[hold[1]].append(bag)
    return dd

In [8]:
may_be_inside(rules)['faded black']

['wavy turquoise', 'muted purple', 'muted blue']

In [9]:
def eventually_may_hold(bag, *, rules):
    allowed = may_be_inside(rules)
    stack = [bag]
    seen = set()
    while stack:
        top = stack.pop()
        seen.add(top)
        for kind in allowed[top]:
            stack.append(kind)
    seen.remove(bag)  # don't want to include our original target bag
    return seen

In [10]:
len(eventually_may_hold('shiny gold', rules=rules))

126

### Solution to Part 2

In [11]:
def must_hold(rules):
    dd = collections.defaultdict(list)
    for bag, holds in parse_rules(rules):
        for hold in holds:
            dd[bag].append(hold)
    return dd

In [12]:
holds = must_hold(rules)
holds['shiny gold']

[('3', 'vibrant orange'), ('3', 'plaid silver')]

In [13]:
def count_bags(bag, *, holds):
    count = 1
    for hold in holds[bag]:
        n = int(hold[0])
        kind = hold[1]
        count += (n * count_bags(kind, holds=holds))
    return count

In [14]:
count_bags('shiny gold', holds=holds)
# answer is this minus 1 (includes the 'shiny gold' bag too)

220150